In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from peft import LoraConfig, TaskType, get_peft_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoModel,
    AutoTokenizer,
    CLIPModel,
    CLIPProcessor,
    Trainer,
    TrainingArguments,
    pipeline,
)

/net/tscratch/people/plgmilosz03/conda-envs/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from huggingface_hub import login
login(token="...")

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
model_name = "speakleash/Bielik-1.5B-v3"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, output_attentions=False, output_hidden_states=False, cache_dir="...",
).to(device)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at speakleash/Bielik-1.5B-v3 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
class HateSpeechDataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_length=512):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    
    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self, idx):
        sentence = str(self.sentences[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            sentence,
            truncation=True,           
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [6]:
df = pd.read_csv("hate_train.csv", nrows=2500)
df.head()

,sentence,label
0,Dla mnie faworytem do tytułu będzie Cracovia. ...,0
1,@anonymized_account @anonymized_account Brawo ...,0
2,"@anonymized_account @anonymized_account Super,...",0
3,@anonymized_account @anonymized_account Musi. ...,0
4,"Odrzut natychmiastowy, kwaśna mina, mam problem",0


In [7]:
sentences = df['sentence'].tolist()
labels = df['label'].tolist()

In [8]:
X_train, X_val, y_train, y_val = train_test_split(
  sentences, 
  labels, 
  test_size=0.2,
  random_state=42,
  stratify=labels
)

In [9]:
train_dataset = HateSpeechDataset(sentences=X_train, labels=y_train, tokenizer=tokenizer)
eval_dataset = HateSpeechDataset(sentences=X_val, labels=y_val, tokenizer=tokenizer)

In [10]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {trainable_params:,}")

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    logging_dir="./logs",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)


Number of trainable parameters: 1,547,358,720


In [11]:
trainer.train()


Step,Training Loss
10,0.006500
20,6.594000
30,2.080700
40,1.285800
50,0.913900
60,2.693200
70,1.380400
80,1.361100
90,0.919400
100,1.581800


TrainOutput(global_step=1000, training_loss=0.7194834705221002, metrics={'train_runtime': 753.6488, 'train_samples_per_second': 2.654, 'train_steps_per_second': 1.327, 'total_flos': 9204982087680000.0, 'train_loss': 0.7194834705221002, 'epoch': 1.0})

In [12]:
eval_dataloader = torch.utils.data.DataLoader(eval_dataset, batch_size=2, shuffle=False)

all_predictions = []
all_labels = []

model.eval()

# Evaluate without gradient computation
with torch.no_grad():
    for batch in eval_dataloader:
        # Move batch to device
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Get model predictions
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=-1)

        # Store predictions and labels
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_predictions)
print(f"Model Accuracy: {accuracy:.4f}")

Model Accuracy: 0.9220
